In [1]:
from scripts import setup_environment

setup_environment()

In [2]:
import textwrap

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

# System message templates (priming)
system_templates = {
    "default": "You are an AI assistant.",
    "default_improved": "You are an AI assistant specialized in solving lateral thinking questions. You will receive a question with multiple choices and must determine the correct answer using logical reasoning and problem-solving techniques.",
    "step_by_step": "You are a logical problem solver. Break down the question systematically, analyze each answer choice step by step, eliminate incorrect options, and select the best answer.",
    "creative": "You are a lateral thinker. Approach each question with flexible reasoning, exploring unconventional yet valid interpretations before selecting the best answer.",
    "elimination": "You are a strategic reasoner. First, identify and eliminate incorrect answer choices. Then, select the most logical remaining option.",
    "metaphor": "You are skilled in abstract reasoning. Consider both literal and metaphorical meanings in the question and choices before selecting the most insightful answer.",
    "confidence": "You are an analytical decision-maker. Assess the likelihood of correctness for each choice, score them internally, and select the answer with the highest confidence.",
    "perspective_shift": "You are a multi-perspective analyst. Evaluate the question from different angles, considering alternative interpretations before determining the best answer.",
    "common_sense": "You balance logic and practicality. Apply both structured reasoning and real-world common sense to determine the most reasonable answer.",
    "assumption_challenge": "You are a critical thinker. Identify and question hidden assumptions in the question and choices before selecting the answer that best challenges or aligns with them.",
    "pattern_matching": "You recognize patterns and relationships. Identify logical structures, recurring themes, or hidden connections in the question and choices before selecting the best answer.",
    "intuitive": "You combine intuition with logic. Generate an initial answer instinctively, then critically evaluate it for logical soundness before finalizing your choice.",
}


def get_system_prompt(template_name: str):
    system_prompt = system_templates[template_name]
    system_prompt = textwrap.dedent(system_prompt)

    system_prompt_template = SystemMessagePromptTemplate.from_template(
        system_prompt, id=template_name
    )
    return system_prompt_template


def get_user_prompt():
    prompt = """
    Please pick the best choice for the brain teaser. Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:

    Question: {question}
    Choice:
    {choices}
    Answer:
    """

    prompt = textwrap.dedent(prompt)

    prompt_template = HumanMessagePromptTemplate.from_template(prompt)
    return prompt_template


def create_prompt_template(
    system_prompt_template_name: str = "default",
):
    system_prompt_template = get_system_prompt(system_prompt_template_name)
    user_prompt_template = get_user_prompt()
    chat_prompt_template = ChatPromptTemplate.from_messages(
        [system_prompt_template, user_prompt_template]
    )

    return chat_prompt_template

In [3]:
from scripts.dataset import BrainteaserDataset

dataset = BrainteaserDataset("data")

In [4]:
import string

from scripts.dataset import RiddleQuestion


def args_generator(riddle_question: RiddleQuestion):
    template_args = {
        "question": riddle_question.question,
        "choices": "\n".join(
            [
                f"({string.ascii_uppercase[j]}) {choice}"
                for j, choice in enumerate(riddle_question.choice_list)
            ]
        ),
    }

    return template_args


chat_prompt_template = create_prompt_template("default")

In [5]:
from scripts.lmm import OllamaModelBuilder
from scripts.executor import Executor

base_url = "http://108.179.129.43:31701"
model_builder = OllamaModelBuilder(base_url)

executor = Executor(
    models=[
        # Llama3.1
        model_builder.build_model("llama3.1:8b"),
        # Llama3.2
        model_builder.build_model("llama3.2:1b"),
        model_builder.build_model("llama3.2:3b"),
        # Phi3.5
        model_builder.build_model("phi3.5:3.8b"),
        # Phi4
        model_builder.build_model("phi4:14b"),
        # Qwen2.5
        model_builder.build_model("qwen2.5:0.5b"),
        model_builder.build_model("qwen2.5:1.5b"),
        model_builder.build_model("qwen2.5:3b"),
        model_builder.build_model("qwen2.5:7b"),
        model_builder.build_model("qwen2.5:14b"),
        model_builder.build_model("qwen2.5:32b"),
        # Gemma2
        model_builder.build_model("gemma2:2b"),
        model_builder.build_model("gemma2:9b"),
        model_builder.build_model("gemma2:27b"),
        # Mistral Nemo
        model_builder.build_model("mistral-nemo:12b"),
    ]
)

2025-02-18 16:05:01,454 - INFO - Initialized executor with 15 models.


In [6]:
test_sample_size = 50
sp_data = dataset.sp[0:test_sample_size]
wp_data = dataset.wp[0:test_sample_size]

In [ ]:
from pathlib import Path

import dill as pickle

total_results = {}

for technique in system_templates:
    chat_prompt_template = create_prompt_template(technique)
    sp_results = await executor.aexecute(
        sp_data,
        chat_prompt_template,
        args_generator,
        dump_to_pickle=True,
        create_checkpoints=True,
        resume_from_checkpoint=True,
        result_file_name=f"test_0/sp_results_{technique}",
    )

    wp_results = await executor.aexecute(
        wp_data,
        chat_prompt_template,
        args_generator,
        dump_to_pickle=True,
        create_checkpoints=True,
        resume_from_checkpoint=True,
        result_file_name=f"test_0/wp_results_{technique}",
    )

    total_results[technique] = {"sp": sp_results, "wp": wp_results}

results_file = Path("results/results_test_1_system-prime-messages.pkl")
results_file.parent.mkdir(parents=True, exist_ok=True)

with results_file.open("wb") as f:
    pickle.dump(total_results, f)

2025-02-18 16:05:01,564 - INFO - Restored results from results file results/test_0/sp_results_default.pkl, skipping execution for this model!
2025-02-18 16:05:01,573 - INFO - Restored results from results file results/test_0/wp_results_default.pkl, skipping execution for this model!
2025-02-18 16:05:01,583 - INFO - Restored results from results file results/test_0/sp_results_default_improved.pkl, skipping execution for this model!
2025-02-18 16:05:01,592 - INFO - Restored results from results file results/test_0/wp_results_default_improved.pkl, skipping execution for this model!
2025-02-18 16:05:01,711 - INFO - Restored results from results file results/test_0/sp_results_step_by_step.pkl, skipping execution for this model!
2025-02-18 16:05:01,722 - INFO - Restored results from results file results/test_0/wp_results_step_by_step.pkl, skipping execution for this model!
2025-02-18 16:05:01,732 - INFO - Restored results from results file results/test_0/sp_results_creative.pkl, skipping exe

## Eval results


In [8]:
import heapq

from scripts.evaluation import eval_model_results


def get_best_prompt_for_each_model(input_data):
    best_prompts = {}

    # Iterate through each model
    for prompt_type, datasets in input_data.items():
        for dataset_type, models in datasets.items():
            # For each model, we need to track its best score
            for model, result in models.items():
                # Initialize the best prompt data structure for this model if not yet created
                if model not in best_prompts:
                    best_prompts[model] = {}

                # Assume eval_results returns a score based on the result data
                score = eval_model_results(result)

                # If this model doesn't have a best score for this dataset yet or if the current score is better
                if (
                    dataset_type not in best_prompts[model]
                    or score > best_prompts[model][dataset_type]["score"]
                ):
                    best_prompts[model][dataset_type] = {
                        "prompt_type": prompt_type,
                        "score": score,
                    }

    # Now best_prompts contains the best prompt type for each model and dataset
    return best_prompts


def get_best_n_prompts_for_each_model(input_data, n=3):
    best_prompts = {}

    # Iterate through each model
    for prompt_type, datasets in input_data.items():
        for dataset_type, models in datasets.items():
            for model, result in models.items():
                # Initialize the best prompt data structure for this model if not yet created
                if model not in best_prompts:
                    best_prompts[model] = {}

                # Calculate the score for the model with the current prompt type and dataset
                score = eval_model_results(result)

                # Initialize the list of prompts for this model and dataset type if not created
                if dataset_type not in best_prompts[model]:
                    best_prompts[model][dataset_type] = []

                # Append the prompt type and score to the list
                best_prompts[model][dataset_type].append(
                    {"prompt_type": prompt_type, "score": score}
                )

    # Now sort the list of prompts for each model and dataset type and keep the top n
    top_n_prompts = {}
    for model, dataset_dict in best_prompts.items():
        top_n_prompts_for_model = {}
        for dataset_type, prompts in dataset_dict.items():
            # Get the top n prompts by sorting the list based on score (highest score first)
            sorted_prompts = heapq.nlargest(n, prompts, key=lambda x: x["score"])
            top_n_prompts_for_model[dataset_type] = sorted_prompts
        top_n_prompts[model] = top_n_prompts_for_model

    return top_n_prompts


# Load the results
results_wrapper = None
with open(results_file, "rb") as f:
    results_wrapper = pickle.load(f)

# Get the best prompt type for each model
best_prompt_types = get_best_prompt_for_each_model(results_wrapper)
print(best_prompt_types)

# Save the best prompt types
with open("results/dict_best_system_prompt_for_models.pkl", "wb") as f:
    pickle.dump(best_prompt_types, f)

{'llama3.1:8b': {'sp': {'prompt_type': 'default_improved', 'score': 62.0}, 'wp': {'prompt_type': 'pattern_matching', 'score': 38.0}}, 'llama3.2:1b': {'sp': {'prompt_type': 'confidence', 'score': 6.0}, 'wp': {'prompt_type': 'confidence', 'score': 12.0}}, 'llama3.2:3b': {'sp': {'prompt_type': 'default', 'score': 32.0}, 'wp': {'prompt_type': 'intuitive', 'score': 24.0}}, 'phi3.5:3.8b': {'sp': {'prompt_type': 'default', 'score': 34.0}, 'wp': {'prompt_type': 'assumption_challenge', 'score': 30.0}}, 'phi4:14b': {'sp': {'prompt_type': 'default', 'score': 86.0}, 'wp': {'prompt_type': 'default_improved', 'score': 66.0}}, 'qwen2.5:0.5b': {'sp': {'prompt_type': 'step_by_step', 'score': 26.0}, 'wp': {'prompt_type': 'step_by_step', 'score': 28.000000000000004}}, 'qwen2.5:1.5b': {'sp': {'prompt_type': 'step_by_step', 'score': 14.000000000000002}, 'wp': {'prompt_type': 'step_by_step', 'score': 20.0}}, 'qwen2.5:3b': {'sp': {'prompt_type': 'intuitive', 'score': 20.0}, 'wp': {'prompt_type': 'intuitive',